<a href="https://colab.research.google.com/github/chris-dewitt/Project/blob/main/GPv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[CentralBankBERT](https://www.sciencedirect.com/science/article/pii/S2405918823000302) -- Here is the paper for the model

Upload Data

In [1]:
import pandas as pd
import numpy as np

fed = pd.read_csv("https://raw.githubusercontent.com/vtasca/fed-statement-scraping/refs/heads/master/communications.csv")
fed = fed[fed.Type == 'Statement'].reset_index()

Clean and Transform Data

In [2]:
import datetime
dates = []
for date in fed.Date:
  year = int(str(date)[:4])
  m = int(str(date)[5:7])
  d = int(str(date)[8:])
  dates.append(datetime.datetime(year, m, d))

In [3]:
fed['Date'] = dates

In [4]:
fed1 = fed.drop(labels=['index', 'Release Date', 'Type'], axis =1)
fed1.head()

,Date,Text
0,2025-03-19,Recent indicators suggest that economic activi...
1,2025-01-29,Recent indicators suggest that economic activi...
2,2024-12-18,Recent indicators suggest that economic activi...
3,2024-11-07,Recent indicators suggest that economic activi...
4,2024-09-18,Recent indicators suggest that economic activi...


In [5]:
def delete_after_phrase(text, phrase):
    index = text.find(phrase)
    if index != -1:
        return text[:index]
    return text

n_txt=[]
phrase = "\n\nFor media inquiries,"
for txt in fed1.Text:
  n_txt.append(delete_after_phrase(txt, phrase))
fed1['Text'] = n_txt

In [6]:
n_txt=[]
phrase = "\n\nVoting for the monetary policy action"
for txt in fed1.Text:
  n_txt.append(delete_after_phrase(txt, phrase))
fed1['Text'] = n_txt

FRED Data Integration

In [7]:
assets = pd.read_csv("/WALCL.csv")
assets.head()

FileNotFoundError: [Errno 2] No such file or directory: '/WALCL.csv'

In [ ]:
assets['observation_date'] = pd.to_datetime(assets['observation_date'])

In [ ]:
fed1['assetDate'] = [min(assets.observation_date, key = lambda date: abs(date - d)) for d in fed1.Date]

In [ ]:
fed_df = pd.merge(fed1, assets, left_on='assetDate', right_on='observation_date', how='left').drop(labels = ['observation_date', 'assetDate'], axis = 1)
fed_df.head()

,Date,Text,WALCL
0,2025-03-19,Recent indicators suggest that economic activi...,6755982
1,2025-01-29,Recent indicators suggest that economic activi...,6818186
2,2024-12-18,Recent indicators suggest that economic activi...,6889332
3,2024-11-07,Recent indicators suggest that economic activi...,6994299
4,2024-09-18,Recent indicators suggest that economic activi...,7109137


In [ ]:
ffr = pd.read_csv('/FEDFUNDS.csv')
ffr['observation_date'] = pd.to_datetime(ffr['observation_date'])
ffr.tail()

,observation_date,FEDFUNDS
843,2024-10-01,4.83
844,2024-11-01,4.64
845,2024-12-01,4.48
846,2025-01-01,4.33
847,2025-02-01,4.33


In [ ]:
fed_df['ffrDate'] = [min(ffr.observation_date, key = lambda date: abs(date - d)) for d in fed_df.Date]

In [ ]:
fed_df2 = pd.merge(fed_df, ffr, left_on='ffrDate', right_on='observation_date', how='left').drop(labels = ['observation_date', 'ffrDate'], axis = 1)
fed_df2.head()

,Date,Text,WALCL,FEDFUNDS
0,2025-03-19,Recent indicators suggest that economic activi...,6755982,4.33
1,2025-01-29,Recent indicators suggest that economic activi...,6818186,4.33
2,2024-12-18,Recent indicators suggest that economic activi...,6889332,4.33
3,2024-11-07,Recent indicators suggest that economic activi...,6994299,4.64
4,2024-09-18,Recent indicators suggest that economic activi...,7109137,4.83


In [ ]:
uer = pd.read_csv('/UNRATE.csv')
uer['observation_date'] = pd.to_datetime(uer['observation_date'])
uer.tail()

,observation_date,UNRATE
297,2024-10-01,4.1
298,2024-11-01,4.2
299,2024-12-01,4.1
300,2025-01-01,4.0
301,2025-02-01,4.1


In [ ]:
fed_df2['uerDate'] = [min(uer.observation_date, key = lambda date: abs(date - d)) for d in fed_df2.Date]

In [ ]:
fed_df3 = pd.merge(fed_df2, uer, left_on='uerDate', right_on='observation_date', how='left').drop(labels = ['observation_date', 'uerDate'], axis = 1)
fed_df3.head()

,Date,Text,WALCL,FEDFUNDS,UNRATE
0,2025-03-19,Recent indicators suggest that economic activi...,6755982,4.33,4.1
1,2025-01-29,Recent indicators suggest that economic activi...,6818186,4.33,4.1
2,2024-12-18,Recent indicators suggest that economic activi...,6889332,4.33,4.0
3,2024-11-07,Recent indicators suggest that economic activi...,6994299,4.64,4.2
4,2024-09-18,Recent indicators suggest that economic activi...,7109137,4.83,4.1


In [ ]:
pay = pd.read_csv('/PAYEMS_CH1.csv')
pay['observation_date'] = pd.to_datetime(pay['observation_date'])
pay.tail()

,observation_date,PAYEMS_CH1
292,2024-10-01,1838
293,2024-11-01,1958
294,2024-12-01,2012
295,2025-01-01,2018
296,2025-02-01,1947


In [ ]:
fed_df3['payDate'] = [min(pay.observation_date, key = lambda date: abs(date - d)) for d in fed_df3.Date]

In [ ]:
fed_df_pay = pd.merge(fed_df3, pay, left_on='payDate', right_on='observation_date', how='left').drop(labels = ['observation_date', 'payDate'], axis = 1)
fed_df_pay.head()

,Date,Text,WALCL,FEDFUNDS,UNRATE,cpiDate,PAYEMS_CH1
0,2025-03-19,Recent indicators suggest that economic activi...,6755982,4.33,4.1,2025-02-01,1947
1,2025-01-29,Recent indicators suggest that economic activi...,6818186,4.33,4.1,2025-02-01,1947
2,2024-12-18,Recent indicators suggest that economic activi...,6889332,4.33,4.0,2025-01-01,2018
3,2024-11-07,Recent indicators suggest that economic activi...,6994299,4.64,4.2,2024-11-01,1958
4,2024-09-18,Recent indicators suggest that economic activi...,7109137,4.83,4.1,2024-10-01,1838


In [ ]:
cpi = pd.read_csv('/CORESTICKM679SFRBATL.csv')
cpi['observation_date'] = pd.to_datetime(cpi['observation_date'])
cpi.tail()

,observation_date,CORESTICKM679SFRBATL
297,2024-10-01,3.643250
298,2024-11-01,3.358040
299,2024-12-01,2.978223
300,2025-01-01,3.437027
301,2025-02-01,3.537460


In [ ]:
fed_df3['cpiDate'] = [min(cpi.observation_date, key = lambda date: abs(date - d)) for d in fed_df3.Date]

In [ ]:
fed_df4 = pd.merge(fed_df3, cpi, left_on='cpiDate', right_on='observation_date', how='left').drop(labels = ['observation_date', 'cpiDate'], axis = 1).reset_index()
fed_df4.head()

,index,Date,Text,WALCL,FEDFUNDS,UNRATE,CORESTICKM679SFRBATL
0,0,2025-03-19,Recent indicators suggest that economic activi...,6755982,4.33,4.1,3.537460
1,1,2025-01-29,Recent indicators suggest that economic activi...,6818186,4.33,4.1,3.537460
2,2,2024-12-18,Recent indicators suggest that economic activi...,6889332,4.33,4.0,3.437027
3,3,2024-11-07,Recent indicators suggest that economic activi...,6994299,4.64,4.2,3.358040
4,4,2024-09-18,Recent indicators suggest that economic activi...,7109137,4.83,4.1,3.643250


Sentiment Classifier

In [ ]:
from transformers import pipeline

sentiment_classifier = pipeline("text-classification", model="Moritz-Pfeifer/CentralBankRoBERTa-sentiment-classifier")


Device set to use cuda:0


In [ ]:
# Not Needed For Analysis
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Moritz-Pfeifer/CentralBankRoBERTa-sentiment-classifier")

import re

def clean_text(text):
    text = re.sub(r"[^\w\s.,!?]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text


def split_text_into_chunks(text, max_tokens=512, overlap=50):
    text=clean_text(text)
    tokens = tokenizer.tokenize(text)
    if not tokens:
        return []  # Prevent empty chunk errors

    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk = tokens[i : i + max_tokens]  # Ensure it doesn’t exceed the model’s limit
        chunk_text = tokenizer.convert_tokens_to_string(chunk)

        if chunk_text.strip():  # Prevent empty strings
            chunks.append(chunk_text)

    return chunks


In [ ]:
agent_classifier = pipeline("text-classification", model="Moritz-Pfeifer/CentralBankRoBERTa-agent-classifier")

Device set to use cuda:0


In [ ]:
fed_df4.Text[150]

"The Federal Open Market Committee decided today to keep its target for the federal funds rate at 5-1/4 percent.\nEconomic growth was moderate during the first half of the year. Financial markets have been volatile in recent weeks, credit conditions have become tighter for some households and businesses, and the housing correction is ongoing. Nevertheless, the economy seems likely to continue to expand at a moderate pace over coming quarters, supported by solid growth in employment and incomes and a robust global economy.\nReadings on core inflation have improved modestly in recent months. However, a sustained moderation in inflation pressures has yet to be convincingly demonstrated. Moreover, the high level of resource utilization has the potential to sustain those pressures.\nAlthough the downside risks to growth have increased somewhat, the Committee's predominant policy concern remains the risk that inflation will fail to moderate as expected. Future policy adjustments will depend 

In [ ]:
agent_classifier(fed_df4.Text[212],truncation=True, max_length=512)

[{'label': 'Central Bank', 'score': 0.9210191965103149}]

In [ ]:
# szes =[]
# for txt in fed_df4.Text:
#   chnk = split_text_into_chunks(txt)
#   szes.append(len(chnk))
# szes == 3

In [ ]:
scrs = []

for i, text in enumerate(fed_df4.Text):
    chnks = split_text_into_chunks(text)
    if len(chnks) > 1:
      txt1 = chnks.pop(0)
      scrs.append(sentiment_classifier(txt1, truncation=True, max_length=512))
    else:
      scrs.append(sentiment_classifier(text, truncation=True, max_length=512))
    print(f"{i}/213")

0/213
1/213
2/213
3/213
4/213
5/213
6/213
7/213
8/213
9/213
10/213
11/213
12/213
13/213
14/213
15/213
16/213
17/213
18/213
19/213
20/213
21/213
22/213
23/213
24/213
25/213
26/213
27/213
28/213
29/213
30/213
31/213
32/213
33/213
34/213
35/213
36/213
37/213
38/213
39/213
40/213
41/213
42/213
43/213
44/213
45/213
46/213
47/213
48/213
49/213
50/213
51/213
52/213
53/213
54/213
55/213
56/213
57/213
58/213
59/213
60/213
61/213
62/213
63/213
64/213
65/213
66/213
67/213
68/213
69/213
70/213
71/213
72/213
73/213
74/213
75/213
76/213
77/213
78/213
79/213
80/213
81/213
82/213
83/213
84/213
85/213
86/213
87/213
88/213
89/213
90/213
91/213
92/213
93/213
94/213
95/213
96/213
97/213
98/213
99/213
100/213
101/213
102/213
103/213
104/213
105/213
106/213
107/213
108/213
109/213
110/213
111/213
112/213
113/213
114/213
115/213
116/213
117/213
118/213
119/213
120/213
121/213
122/213
123/213
124/213
125/213
126/213
127/213
128/213
129/213
130/213
131/213
132/213
133/213
134/213
135/213
136/213
137/213
138/21

In [ ]:
fed_df4['Sentiment'] = [scr[0]['label'] for scr in scrs]
fed_df4['Confidence'] = [scr[0]['score'] for scr in scrs]

SP500 ETF Load

In [ ]:
import yfinance as yf
min_date = min(fed_df4['Date'])
spy = yf.download('VIOO', start=min_date)

[*********************100%***********************]  1 of 1 completed


In [ ]:
op_c = []
cl_3 = []
for date in fed_df4['Date']:
  three_ahead = date + np.timedelta64(3, 'D')
  if spy[spy.index == date].Open.values:
    op_c.append(spy[spy.index == date].Open.values)
    if spy[spy.index == (date + np.timedelta64(3, 'D'))].Close.values:
      cl_3.append(spy[spy.index == date + np.timedelta64(3, 'D')].Close.values)
    else:
      if three_ahead.dayofweek == 5:
        if spy[spy.index == date + np.timedelta64(5, 'D')].Close.values:
          cl_3.append(spy[spy.index == date + np.timedelta64(5, 'D')].Close.values)
        else:
          cl_3.append(spy[spy.index == date + np.timedelta64(6, 'D')].Close.values)
      elif three_ahead.dayofweek == 6:
        if spy[spy.index == date + np.timedelta64(4, 'D')].Close.values:
          cl_3.append(spy[spy.index == date + np.timedelta64(4, 'D')].Close.values)
        else:
          cl_3.append(spy[spy.index == date + np.timedelta64(4, 'D')].Close.values)
      else:
        if spy[spy.index == date + np.timedelta64(2, 'D')].Close.values:
          cl_3.append(spy[spy.index == date + np.timedelta64(2, 'D')].Close.values)
        else:
          cl_3.append(spy[spy.index == date + np.timedelta64(2, 'D')].Close.values)

  else:
    op_c.append(spy[spy.index == date + np.timedelta64(1, 'D')].Open.values)
    if spy[spy.index == date + np.timedelta64(4, 'D')].Close.values:
      cl_3.append(spy[spy.index == date + np.timedelta64(4, 'D')].Close.values)
    else:
      if three_ahead.dayofweek == 5:
        cl_3.append(spy[spy.index == date + np.timedelta64(5, 'D')].Close.values)
      elif three_ahead.dayofweek == 6:
        cl_3.append(spy[spy.index == date + np.timedelta64(4, 'D')].Close.values)
      else:
        cl_3.append(spy[spy.index == date + np.timedelta64(4, 'D')].Close.values)

<ipython-input-172-0873f35b0aac>:7: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if spy[spy.index == (date + np.timedelta64(3, 'D'))].Close.values:
<ipython-input-172-0873f35b0aac>:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if spy[spy.index == date + np.timedelta64(5, 'D')].Close.values:
<ipython-input-172-0873f35b0aac>:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if spy[spy.index == date].Open.values:
<ipython-input-172-0873f35b0aac>:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result 

In [ ]:
fed_df4['Open'] = [x for x in op_c]
fed_df4['3_day_close'] =[x if x else 0 for x in cl_3]
fed1 = fed_df4[fed_df4['3_day_close'] != 0]
fed_df4 = fed1[fed1['Open'] != 0]

<ipython-input-173-1dd0ba0d4d0a>:2: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  fed_df4['3_day_close'] =[x if x else 0 for x in cl_3]


In [ ]:
fed_df4['3_day_close'] = [x[0][0] for x in fed_df4['3_day_close']]
fed_df4['Open'] = [x[0][0] for x in fed_df4['Open']]

In [ ]:
fed_df4['3d_return'] = [(tdc - o) / o for o, tdc in zip(fed_df4['Open'], fed_df4['3_day_close'])]

In [ ]:
res1 = []
for x in fed_df4['3d_return']:
  if x >= 0.0:
    res1.append(1)
  else:
    res1.append(0)

In [ ]:
fed_df4['result'] = res1

Final Transform

In [ ]:
fed_df4['Sentiment_Bool']=[1 if s[0] == 'positive' and s[1] > 0.7  else 0 for s in fed_df4[['Sentiment', 'Confidence']].values]

In [ ]:
# Change Confusing Column Names
fed_final = fed_df4.rename(columns={'CORESTICKM679SFRBATL': 'cpi', 'WALCL': 'assets'}).drop(labels=['index'], axis=1)

In [ ]:
fed_final

,Date,Text,assets,FEDFUNDS,UNRATE,cpi,Sentiment,Confidence,Open,3_day_close,3d_return,result,Sentiment_Bool
0,2025-03-19,Recent indicators suggest that economic activi...,6755982,4.33,4.1,3.537460,positive,0.509480,97.110001,99.709999,0.026774,1,0
1,2025-01-29,Recent indicators suggest that economic activi...,6818186,4.33,4.1,3.537460,positive,0.558265,109.129997,107.430000,-0.015578,0,0
2,2024-12-18,Recent indicators suggest that economic activi...,6889332,4.33,4.0,3.437027,positive,0.576250,111.357237,106.580002,-0.042900,0,0
3,2024-11-07,Recent indicators suggest that economic activi...,6994299,4.64,4.2,3.358040,positive,0.558645,113.239467,115.180824,0.017144,1,0
4,2024-09-18,Recent indicators suggest that economic activi...,7109137,4.83,4.1,3.643250,positive,0.567516,105.069991,106.272247,0.011442,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2011-03-15,Information received since the Federal Open Ma...,2583046,0.14,9.0,1.949567,positive,0.641854,25.913215,26.318968,0.015658,1,0
116,2011-01-26,Information received since the Federal Open Ma...,2443527,0.16,9.0,1.699002,negative,0.584768,25.892080,26.082275,0.007346,1,0
117,2010-12-14,Information received since the Federal Open Ma...,2385754,0.18,9.3,1.047836,negative,0.574312,25.645693,25.616278,-0.001147,0,0
118,2010-11-03,Information received since the Federal Open Ma...,2300353,0.19,9.8,1.110615,negative,0.672062,23.565637,24.355633,0.033523,1,0


In [ ]:
# Standardize Data for Logistic Reg and NB
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tofit = fed_final[['assets', 'FEDFUNDS', 'UNRATE', 'cpi']]
X_scaled = scaler.fit_transform(X_tofit)
y = fed_final['result']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tofit, y, test_size=0.3, random_state=42)

Logistic Regression Model

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Using KFolds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

log_reg = LogisticRegression(C=10)
log_scores = cross_val_score(log_reg, X_tofit, y, cv=kf, scoring='accuracy')
print("Logistic Regression CV Accuracy Scores:", log_scores)
print("Logistic Regression Mean Accuracy:", np.mean(log_scores))

log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred))

Logistic Regression CV Accuracy Scores: [0.54166667 0.375      0.5        0.45833333 0.60869565]
Logistic Regression Mean Accuracy: 0.4967391304347826
0.6388888888888888 0.7333333333333333 0.55


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb_scores = cross_val_score(nb, X_tofit, y, cv=kf, scoring='accuracy')
print("\nNaive Bayes CV Accuracy Scores:", nb_scores)
print("Naive Bayes Mean Accuracy:", np.mean(nb_scores))


Naive Bayes CV Accuracy Scores: [0.45833333 0.5        0.45833333 0.41666667 0.60869565]
Naive Bayes Mean Accuracy: 0.4884057971014492


In [ ]:
nb.fit(X_train, y_train)

GaussianNB()

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = nb.predict(X_test)
ac_sc = accuracy_score(y_true=y_test, y_pred=y_pred)
print(ac_sc)

0.5277777777777778


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
kmeans = KMeans(n_clusters=3, random_state=0, n_init='auto')
kmeans.fit(X_tofit)

# Predict the cluster labels
labels = kmeans.predict(X_tofit)

# Get the cluster centers
centers = kmeans.cluster_centers_

# # Plot the data points and cluster centers
# plt.scatter(X_tofit[:, 0], X_tofit[:, 1], c=labels, cmap='viridis')
# plt.scatter(centers[:, 0], centers[:, 1], marker='x', s=200, color='red')
# plt.show()

In [ ]:
scaler = StandardScaler()
X_tofit = fed_final[['assets', 'FEDFUNDS', 'UNRATE', 'cpi', 'Sentiment_Bool']]
X_scaled = scaler.fit_transform(X_tofit)
y = fed_final['3d_return']

X_train, X_test, y_train, y_test = train_test_split(X_tofit, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline

degree = 3
model = Lasso(alpha=.01)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print(mse*100)

0.14824087835747612


In [ ]:
print(np.mean((y_pred - y_test)*100))

-0.3637035510473967


In [ ]:
from sklearn.metrics import r2_score
r2 = model.score(X_test, y_test)
print(r2)

-0.02632944654518532
